# Senegal Assessment Notebook

Notebook to perform assessment of CHM and Land Cover results over senegal.

## TODO

- Add marker of imagery based on year options
- Add marker of imagery and zoom into overlay raster
- Add marker of imagery and push on and off landcover layers
- Marker with histogram, super-imposing the histograms
- Plot multiple images, filter by year

## Define data paths

Here we define data path with regexs.

In [1]:
import os
import re
import tempfile
import folium
import numpy as np
import rasterio as rio
import ipywidgets as widgets
import branca.colormap as cm
import matplotlib.pyplot as plt
import pandas as pd
import altair as alt
import json

from glob import glob
from pathlib import Path
from folium import plugins
from pyproj import Transformer 
from rasterio.warp import calculate_default_transform, reproject, Resampling

os.environ['LOCALTILESERVER_CLIENT_PREFIX'] = \
    f"{os.environ['JUPYTERHUB_SERVICE_PREFIX'].lstrip('/')}/proxy/{{port}}"

from localtileserver import get_folium_tile_layer, TileClient

In [2]:
data_bands = [5, 2, 7]
data_dir = '/adapt/nobackup/projects/3sl/data/Tappan/*_data.tif'
landcover_dir = '/adapt/nobackup/projects/ilab/projects/Senegal/3sl/products/land_cover/dev/trees.v2/Tappan/*.tif'
chm_dir = '/adapt/nobackup/projects/ilab/projects/Senegal/CNN_CHM/v4/output/*.tif'

Then, we generate lists out of the files available on the dirs.

In [3]:
data_filenames = sorted(glob(data_dir))
landcover_filenames = sorted(glob(landcover_dir))
chm_filenames = sorted(glob(chm_dir))
assert len(data_filenames) == len(landcover_filenames) \
    and len(landcover_filenames) == len(chm_filenames), \
    'Not all layers have the same number of file'
print(f'Loading {len(data_filenames)} filenames.')

Loading 272 filenames.


In [4]:
filenames_metadata = dict()
for d, l, c in zip(data_filenames, landcover_filenames, chm_filenames):
    
    #raster_date = re.search(r'\d{4}\d{2}\d{2}', d).group(0)
    raster_date = re.search(r'\d{4}', d).group(0)
    raster_id = re.search(r'Tappan\d{2}', d).group(0)
    
    #print(filenames_metadata.keys())
    if raster_date not in filenames_metadata:
        filenames_metadata[raster_date] = [
            {
                'data_layer': d,
                'landcover_layer': l,
                'chm_layer': c
            }
        ]
    else:
        filenames_metadata[raster_date].append(
            {
                'data_layer': d,
                'landcover_layer': l,
                'chm_layer': c
            }
        )
    #print(raster_date.group(0), raster_id.group(0), d)


filenames_metadata = dict(sorted(filenames_metadata.items()))

## Define basemaps for later use

Here we define basemaps for use with folium on the backend.

In [5]:
basemaps = {
       'Google Terrain': folium.TileLayer(
        tiles = 'https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}',
        attr = 'Google',
        name = 'Google Terrain',
        overlay = False,
        control = True
       ),
        'basemap_gray': folium.TileLayer(
            tiles="http://services.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}",
            opacity=1,
            name="World gray basemap",
            attr="ESRI",
            overlay=False
        ),
        'Imagery': folium.TileLayer(
            tiles='https://services.arcgisonline.com/arcgis/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
            opacity=1,
            name="World Imagery",
            attr="ESRI",
            overlay=False
        ),
        'ESRINatGeo': folium.TileLayer(
            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/NatGeo_World_Map/MapServer/tile/{z}/{y}/{x}',
            opacity=1,
            name='ESRI NatGeo',
            attr='ESRI',
            overlay=False
        )
}

In [6]:
# Client - initial client to localize zoom
data_client = TileClient(data_filenames[0])

# Map the Layers
folium_map = folium.Map(
    location=data_client.center(),
    zoom_start=16,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google'
)

# TODO: feature groups can be further divided in years
feature_group_data = folium.FeatureGroup(name='EVHR TOA')
feature_group_landcover = folium.FeatureGroup(name='Trees')
feature_group_chm = folium.FeatureGroup(name='CHM')

for data_filename, landcover_filename, chm_filename in zip(
    data_filenames[:1], landcover_filenames[:1], chm_filenames[:1]):
    
    # client for data tile
    data_client = TileClient(data_filename)
    
    # client for landcover tile
    landcover_client = TileClient(landcover_filename)
    
    # client for chm tile
    chm_client = TileClient(chm_filename)
    
    # group all data layers into one single control
    feature_group_data.add_child(
        get_folium_tile_layer(data_client))
    
    # group all landcover layers into one single control
    feature_group_landcover.add_child(
        get_folium_tile_layer(landcover_client, cmap='Greens', opacity=0.6))
    
    # group all chm layers into one single control
    feature_group_chm.add_child(
        get_folium_tile_layer(chm_client, cmap='Greens', opacity=0.6))
    
    # add a marker per location
    folium.Marker(
        data_client.center(), popup=Path(data_filename).stem).add_to(folium_map)

folium_map.add_child(feature_group_data)
folium_map.add_child(feature_group_landcover)
folium_map.add_child(feature_group_chm)

#popup = folium.Popup()
#folium.VegaLite(json.loads(chart.to_json()), width="100%", height="100%").add_to(popup)

#folium.Marker(
#    [
#        data_filenames_bounds['center'][1],
#        data_filenames_bounds['center'][0]
#    ],
#).add_to(folium_map)

folium_map.add_child(plugins.Fullscreen())
folium_map.add_child(plugins.Geocoder())
folium_map.add_child(plugins.MousePosition())
folium_map.add_child(folium.LayerControl())

folium_map

## Define Generic Help Functions

In this section we transform the data and plot.

In [7]:
"""
# Client - initial client to localize zoom
data_client = TileClient(data_filenames[0])

# Map the Layers
folium_map = folium.Map(
    location=data_client.center(),
    zoom_start=16,
    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',
    attr='Google'
)

# Iterate over each item for specific dates
for date_key in filenames_metadata:

    print(f'Processed year: {date_key}')
    
    # Divide feature groups into year
    feature_group_data = folium.FeatureGroup(name=f'EVHR TOA - {date_key}')
    feature_group_landcover = folium.FeatureGroup(name=f'Trees - {date_key}')
    feature_group_chm = folium.FeatureGroup(name=f'CHM - {date_key}')

    # iterate over data files
    for raster_id in filenames_metadata[date_key]:
        
        # client for data tile
        data_client = TileClient(raster_id['data_layer'])
    
        # client for landcover tile
        landcover_client = TileClient(raster_id['landcover_layer'])

        # client for chm tile
        chm_client = TileClient(raster_id['chm_layer'])
    
        # group all data layers into one single control
        feature_group_data.add_child(
            get_folium_tile_layer(data_client, show=False))
    
        # group all landcover layers into one single control
        feature_group_landcover.add_child(
            get_folium_tile_layer(landcover_client, cmap='Greens', opacity=0.6, show=False))
    
        # group all chm layers into one single control
        feature_group_chm.add_child(
            get_folium_tile_layer(chm_client, cmap='Greens', opacity=0.6, show=False))
    
        # add a marker per location
        folium.Marker(
            data_client.center(), popup=Path(raster_id['data_layer']).stem
        ).add_to(folium_map)
    
    folium_map.add_child(feature_group_data)
    folium_map.add_child(feature_group_landcover)
    folium_map.add_child(feature_group_chm)

folium_map.add_child(plugins.Fullscreen())
folium_map.add_child(plugins.Geocoder())
folium_map.add_child(plugins.MousePosition())
folium_map.add_child(folium.LayerControl())

folium_map
"""

"\n# Client - initial client to localize zoom\ndata_client = TileClient(data_filenames[0])\n\n# Map the Layers\nfolium_map = folium.Map(\n    location=data_client.center(),\n    zoom_start=16,\n    tiles='https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}',\n    attr='Google'\n)\n\n# Iterate over each item for specific dates\nfor date_key in filenames_metadata:\n\n    print(f'Processed year: {date_key}')\n    \n    # Divide feature groups into year\n    feature_group_data = folium.FeatureGroup(name=f'EVHR TOA - {date_key}')\n    feature_group_landcover = folium.FeatureGroup(name=f'Trees - {date_key}')\n    feature_group_chm = folium.FeatureGroup(name=f'CHM - {date_key}')\n\n    # iterate over data files\n    for raster_id in filenames_metadata[date_key]:\n        \n        # client for data tile\n        data_client = TileClient(raster_id['data_layer'])\n    \n        # client for landcover tile\n        landcover_client = TileClient(raster_id['landcover_layer'])\n\n        # client fo